In [1]:
import pandas as pd
import numpy as np
import requests
import json
from sklearn.cluster import KMeans
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
participants = pd.read_csv('participants.csv')
participants = participants.loc[participants['gameDuration'] > 1500]

C:\Users\zenet\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (55,56) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
participants = participants.replace(True, 1).replace(False, 0)

In [4]:
def renameLanes(df):
    lanes = ['B_BOT1', 'B_BOT2', 'B_JG', 'B_MID', 'B_TOP', 'R_BOT1', 'R_BOT2', 'R_JG', 'R_MID', 'R_TOP']
    list = []
    i = 0
    for l in df:
        list.append(lanes[i])
        i += 1
        i = i % 10
    return list

In [5]:
def renameLanes2(df):
    lanes = [ 'R_BOT1', 'R_BOT2', 'R_JG', 'R_MID', 'R_TOP', 'B_BOT1', 'B_BOT2', 'B_JG', 'B_MID', 'B_TOP']
    list = []
    i = 0
    for l in df:
        list.append(lanes[i])
        i += 1
        i = i % 10
    return list

In [6]:
def winLanes(tid, win):
    list = []
    i = 0
    for t, w in zip(tid, win):
        if(t == 200):
            if(w == 1):
                list.append(1)
            else:
                list.append(2)
        else:
            if(w == 1):
                list.append(2)
            else:
                list.append(1)
        i += 1
    return list

In [7]:
def winLanes2(tid, win):
    list = []
    i = 0
    for t, w in zip(tid, win):
        if(t == 200):
            if(w == 1):
                list.append(2)
            else:
                list.append(1)
        else:
            if(w == 1):
                list.append(1)
            else:
                list.append(2)
        i += 1
    return list

In [8]:
gpby = participants.groupby(['gameId', 'teamId', 'lane', 'win'])['championId'].apply(list).reset_index(name='championPlayed')
gpby = gpby.explode('championPlayed')

gpby['lane'] = renameLanes(gpby['lane'])
gpby['winNew'] = winLanes(gpby['teamId'], gpby['win'])
gpby

,gameId,teamId,lane,win,championPlayed,winNew
0,2191623268,100,B_BOT1,1.0,145,2
0,2191623268,100,B_BOT2,1.0,555,2
1,2191623268,100,B_JG,1.0,245,2
2,2191623268,100,B_MID,1.0,84,2
3,2191623268,100,B_TOP,1.0,92,2
...,...,...,...,...,...,...
37630,2259100011,200,R_BOT1,0.0,117,2
37631,2259100011,200,R_BOT2,0.0,29,2
37631,2259100011,200,R_JG,0.0,36,2
37632,2259100011,200,R_MID,0.0,43,2


In [9]:
gpby2 = participants.groupby(['gameId', 'teamId', 'lane', 'win'])['championId'].apply(list).reset_index(name='championPlayed')
gpby2 = gpby2.explode('championPlayed')

gpby2['lane'] = renameLanes2(gpby2['lane'])
gpby2['winNew'] = winLanes2(gpby2['teamId'], gpby2['win'])
gpby2

,gameId,teamId,lane,win,championPlayed,winNew
0,2191623268,100,R_BOT1,1.0,145,1
0,2191623268,100,R_BOT2,1.0,555,1
1,2191623268,100,R_JG,1.0,245,1
2,2191623268,100,R_MID,1.0,84,1
3,2191623268,100,R_TOP,1.0,92,1
...,...,...,...,...,...,...
37630,2259100011,200,B_BOT1,0.0,117,1
37631,2259100011,200,B_BOT2,0.0,29,1
37631,2259100011,200,B_JG,0.0,36,1
37632,2259100011,200,B_MID,0.0,43,1


In [10]:
gpby_table = gpby.pivot_table(values=['championPlayed'], index=['gameId', 'winNew'], columns='lane', aggfunc='first')
gpby_table

championPlayed                                             \
lane                      B_BOT1 B_BOT2 B_JG B_MID B_TOP R_BOT1 R_BOT2 R_JG   
gameId     winNew                                                             
2191623268 2                 145    555  245    84    92    157    147   79   
2194605998 1                  15    876  555   236    41    147     81  245   
2194934974 2                  79    145  245   103   150    111    236  421   
2196045435 1                 526    145  421   157   150     12     67   23   
2196714186 2                 526    234  360   245    19    236    555  121   
...                          ...    ...  ...   ...   ...    ...    ...  ...   
2259080027 2                  43     18  421    56   517    518     81  121   
2259084673 1                 555      2   96   122    43    110    235  106   
2259085614 2                 350     67   77   103    64     12    110  104   
2259085950 1                 360    117   77    84    14    145    267  106   
2259100011 2                 235     99   78    26   516    117     29   36   

                               
lane              R_MID R_TOP  
gameId     winNew              
2191623268 2         55   223  
2194605998 1        238   150  
2194934974 2          7   126  
2196045435 1        245   266  
2196714186 2        134   122  
...                 ...   ...  
2259080027 2        236    41  
2259084673 1         38   516  
2259085614 2         61   126  
2259085950 1        517    31  
2259100011 2         43   164  

[5075 rows x 10 columns]

In [11]:
gpby_table2 = gpby2.pivot_table(values=['championPlayed'], index=['gameId', 'winNew'], columns='lane', aggfunc='first')
gpby_table2

championPlayed                                             \
lane                      B_BOT1 B_BOT2 B_JG B_MID B_TOP R_BOT1 R_BOT2 R_JG   
gameId     winNew                                                             
2191623268 1                 157    147   79    55   223    145    555  245   
2194605998 2                 147     81  245   238   150     15    876  555   
2194934974 1                 111    236  421     7   126     79    145  245   
2196045435 2                  12     67   23   245   266    526    145  421   
2196714186 1                 236    555  121   134   122    526    234  360   
...                          ...    ...  ...   ...   ...    ...    ...  ...   
2259080027 1                 518     81  121   236    41     43     18  421   
2259084673 2                 110    235  106    38   516    555      2   96   
2259085614 1                  12    110  104    61   126    350     67   77   
2259085950 2                 145    267  106   517    31    360    117   77   
2259100011 1                 117     29   36    43   164    235     99   78   

                               
lane              R_MID R_TOP  
gameId     winNew              
2191623268 1         84    92  
2194605998 2        236    41  
2194934974 1        103   150  
2196045435 2        157   150  
2196714186 1        245    19  
...                 ...   ...  
2259080027 1         56   517  
2259084673 2        122    43  
2259085614 1        103    64  
2259085950 2         84    14  
2259100011 1         26   516  

[5075 rows x 10 columns]

In [12]:
df1 = gpby_table['championPlayed']
df2 = gpby_table2['championPlayed']

In [13]:
df1

,lane,B_BOT1,B_BOT2,B_JG,B_MID,B_TOP,R_BOT1,R_BOT2,R_JG,R_MID,R_TOP
gameId,winNew,,,,,,,,,,
2191623268,2,145,555,245,84,92,157,147,79,55,223
2194605998,1,15,876,555,236,41,147,81,245,238,150
2194934974,2,79,145,245,103,150,111,236,421,7,126
2196045435,1,526,145,421,157,150,12,67,23,245,266
2196714186,2,526,234,360,245,19,236,555,121,134,122
...,...,...,...,...,...,...,...,...,...,...,...
2259080027,2,43,18,421,56,517,518,81,121,236,41
2259084673,1,555,2,96,122,43,110,235,106,38,516
2259085614,2,350,67,77,103,64,12,110,104,61,126


In [14]:
df1.reset_index(level='winNew', inplace=True)
df2.reset_index(level='winNew', inplace=True)

In [15]:
frames = [df1,df2]
df = pd.concat(frames, keys=['1', '0'])
df = df.sample(frac=1)
df

lane          winNew  B_BOT1  B_BOT2  B_JG  B_MID  B_TOP  R_BOT1  R_BOT2  \
  gameId                                                                   
0 2237656517       1      81      12   141    134     86     555      18   
  2202116804       1     875     119    30     42     54      15      25   
1 2243363451       2     223     235   120    103     54      81      43   
  2252161983       1      42      18    11     61     90     202      74   
  2255407136       1     555      81   555     81     30      30      34   
...              ...     ...     ...   ...    ...    ...     ...     ...   
  2243993747       2     222      25   350     84     78     412      81   
  2253199370       2      89     145   238    517     27      50     360   
  2257610817       1      96      60   136     25     41     202     164   
  2246721044       1      43      22   120    234     17      12     222   
  2255325848       2     523     111   523    111    523     111      56   

lane          R_JG  R_MID  R_TOP  
  gameId                          
0 2237656517   104    142     39  
  2202116804    35    245     92  
1 2243363451   106    105     68  
  2252161983    45    134    497  
  2255407136    34     64     64  
...            ...    ...    ...  
  2243993747    64     58     68  
  2253199370   121     58     78  
  2257610817   203    517     84  
  2246721044    77      8    126  
  2255325848    56     56      7  

[10150 rows x 11 columns]

In [16]:
clf=RandomForestClassifier(n_estimators=500, max_features='auto')

In [17]:
att = ['B_BOT1', 'B_BOT2', 'B_JG', 'B_MID', 'B_TOP', 'R_BOT1', 'R_BOT2', 'R_JG', 'R_MID', 'R_TOP']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(df[att], df['winNew'], test_size=0.1)

In [19]:
clf.fit(X_train,y_train)

y_pred=clf.predict(X_test)

In [20]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.4935960591133005


In [21]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train,y_train)
predicted= model.predict(X_test)
print("Predicted Value:", predicted)
print("Accuracy:",metrics.accuracy_score(y_test, predicted))

Predicted Value: [2 1 1 ... 2 1 1]
Accuracy: 0.50935960591133


In [22]:
from sklearn.neighbors import KNeighborsClassifier
model2 = KNeighborsClassifier(n_neighbors=2)
model2.fit(X_train,y_train)
predicted2= model2.predict(X_test)
print("Predicted Value:", predicted)
print("Accuracy:",metrics.accuracy_score(y_test, predicted2))

Predicted Value: [2 1 1 ... 2 1 1]
Accuracy: 0.5123152709359606
